<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_02/TS%20-%20W2%20-%2014%20-%20Atividade_Avaliativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Laboratório Prático: Previsão de séries temporais

Seja bem-vindo!

Na tarefa anterior, você teve alguma exposição ao trabalho com dados de séries temporais, mas não usou técnicas de aprendizado de máquina para suas previsões.

Nesta semana, você usará uma rede neural profunda para criar previsões e ver como essa técnica se compara às que você já experimentou. Mais uma vez, todos os dados serão sintéticos.

Vamos começar!

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from dataclasses import dataclass

In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/cert_prof_time_series/class_02/images.zip
!unzip -n -q images.zip

## Gerando os dados


A próxima célula inclui um conjunto de funções auxiliares para gerar e plotar a série temporal:

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Tempo")
    plt.ylabel("Valor")
    plt.grid(False)

def trend(time, slope=0):
    return slope * time

def seasonal_pattern(season_time):
    """Apenas um padrão arbitrário, você pode alterá-lo se desejar"""
    return np.where(season_time < 0.1,
                    np.cos(season_time * 6 * np.pi), 
                    2 / np.exp(9 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repete o mesmo padrão em cada período"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

Você gerará dados de séries temporais muito parecidos com os dos laboratórios passados, mas com algumas diferenças.

**Observe que, desta vez, toda a geração é feita em uma função e as variáveis globais são salvas em uma classe de dados. Isso é feito para evitar o uso do escopo global, como foi feito na anteriormente.**

Se você nunca usou classes de dados antes, elas são apenas classes Python que fornecem uma sintaxe conveniente para armazenar dados. Você pode ler mais sobre elas em [docs](https://docs.python.org/3/library/dataclasses.html). 

In [ ]:
def generate_time_series():
    # A dimensão de tempo ou a coordenada x da série temporal
    time = np.arange(4 * 365 + 1, dtype="float32")

    # A série inicial é apenas uma linha reta com uma interceptação y
    y_intercept = 10
    slope = 0.005
    series = trend(time, slope) + y_intercept

    # Adição de sazonalidade
    amplitude = 50
    series += seasonality(time, period=365, amplitude=amplitude)

    # Adicionando algum ruído
    noise_level = 3
    series += noise(time, noise_level, seed=51)
    
    return time, series


# Salve todas as variáveis "globais" na classe G (G significa global)
@dataclass
class G:
    TIME, SERIES = generate_time_series()
    SPLIT_TIME = 1100
    WINDOW_SIZE = 20
    BATCH_SIZE = 32
    SHUFFLE_BUFFER_SIZE = 1000
    
# Plotar a série gerada
plt.figure(figsize=(10, 6))
plot_series(G.TIME, G.SERIES)
plt.show()

## Dividir os dados

Como você já codificou a função `train_val_split` durante a atividade avaliativa passada, desta vez ela é fornecida para você:

In [ ]:
def train_val_split(time, series, time_step=G.SPLIT_TIME):

    time_train = time[:time_step]
    series_train = series[:time_step]
    time_valid = time[time_step:]
    series_valid = series[time_step:]

    return time_train, series_train, time_valid, series_valid


# Dividir o conjunto de dados
time_train, series_train, time_valid, series_valid = train_val_split(G.TIME, G.SERIES)

## Processamento dos dados

Como você viu nos laboratórios, é possível alimentar os dados para treinamento criando um conjunto de dados com as etapas de processamento apropriadas, como `windowing`, `flattening`, `batching` e `shuffling`. Para isso, complete a função `windowed_dataset` abaixo.

Observe que essa função recebe um `series`, `window_size`, `batch_size` e `shuffle_buffer` e os três últimos têm como padrão os valores "globais" definidos anteriormente.

Não deixe de consultar a [docs](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) sobre o `TF Datasets` se precisar de ajuda.

In [ ]:
def windowed_dataset(series, window_size=G.WINDOW_SIZE, batch_size=G.BATCH_SIZE, shuffle_buffer=G.SHUFFLE_BUFFER_SIZE):
    
    ### INICIE SEU CÓDIGO AQUI
    
    # Criar conjunto de dados a partir da série
    dataset = None
    
    # Percorra o conjunto de dados com janela apropriada
    dataset = None
    
    # Achatar o conjunto de dados
    dataset = None
    
    # Embaralhe-o
    dataset = None
    
    # Divida-o em recursos e rótulos
    dataset = None
    
    # Definas os lotes
    dataset = None
    
    ### TERMINE SEU CÓDIGO AQUI
    
    return dataset

Para testar sua função, você usará um `window_size` de 1, o que significa que você usará cada valor para prever o próximo. Isso para 5 elementos, já que um `batch_size` de 5 é usado e nenhum embaralhamento, já que `shuffle_buffer` está definido como 1.

Com isso, o lote de recursos deve ser idêntico aos primeiros 5 elementos do `series_train` e o lote de rótulos deve ser igual aos elementos 2 a 6 do `series_train`.

In [ ]:
# Teste sua função com janelas de tamanho 1 e sem embaralhamento
test_dataset = windowed_dataset(series_train, window_size=1, batch_size=5, shuffle_buffer=1)

# Obter o primeiro lote do conjunto de dados de teste
batch_of_features, batch_of_labels = next((iter(test_dataset)))

print(f"batch_of_features é do tipo: {type(batch_of_features)}\n")
print(f"batch_of_labels é do tipo: {type(batch_of_labels)}\n")
print(f"batch_of_features tem o formato: {batch_of_features.shape}\n")
print(f"batch_of_labels tem o formato: {batch_of_labels.shape}\n")
print(f"batch_of_features é igual aos cinco primeiros elementos da série: {np.allclose(batch_of_features.numpy().flatten(), series_train[:5])}\n")
print(f"batch_of_labels  é igual aos cinco primeiros rótulos: {np.allclose(batch_of_labels.numpy(), series_train[1:6])}")

**Saída Esperada:**

```
batch_of_features é do tipo: <class 'tensorflow.python.framework.ops.EagerTensor'>

batch_of_labelsé do tipo: <class 'tensorflow.python.framework.ops.EagerTensor'>

batch_of_features tem o formato: (5, 1)

batch_of_labels tem o formato: (5,)

batch_of_features é igual aos cinco primeiros elementos da série: True

batch_of_labels é igual aos cinco primeiros rótulos: True
```

## Definição da arquitetura do modelo

Agora que você tem uma função que processará os dados antes que eles sejam inseridos na rede neural para treinamento, é hora de definir a arquitetura da camada.

Complete a função `create_model` abaixo. Observe que essa função recebe o `window_size`, pois esse será um parâmetro importante para a primeira camada de sua rede.

Dica:
- Você só precisará das camadas `Dense`.
- Não inclua as camadas `Lambda`. Elas não são necessárias e são incompatíveis com o formato `HDF5` que será usado para salvar seu modelo para classificação.
- O treinamento deve ser muito rápido, portanto, se perceber que cada época está demorando mais do que alguns segundos, considere tentar uma arquitetura diferente.

In [ ]:
def create_model(window_size=G.WINDOW_SIZE):

    ### INICIE SEU CÓDIGO AQUI

    model = tf.keras.models.Sequential([ 
        
    ]) 

    model.compile(loss=None,
                  optimizer=None)
    
    ### TERMINE SEU CÓDIGO AQUI

    return model

In [ ]:
# Aplicar o processamento a toda a série de treinamento
dataset = windowed_dataset(series_train)

# Salvar uma instância do modelo
model = create_model()

# Treine
model.fit(dataset, epochs=100)

## Avaliação da previsão

Agora é hora de avaliar o desempenho da previsão. Para isso, você pode usar a função `compute_metrics` que codificou na tarefa anterior:

In [ ]:
def compute_metrics(true_series, forecast):
    
    mse = tf.keras.metrics.mean_squared_error(true_series, forecast).numpy()
    mae = tf.keras.metrics.mean_absolute_error(true_series, forecast).numpy()

    return mse, mae

Neste ponto, apenas o modelo que executará a previsão está pronto, mas você ainda precisa calcular a previsão real. 

Para isso, execute a célula abaixo que usa a função `generate_forecast` para calcular a previsão. Essa função gera o próximo valor com base em um conjunto de pontos `window_size` anteriores para cada ponto no conjunto de validação.

In [ ]:
def generate_forecast(series=G.SERIES, split_time=G.SPLIT_TIME, window_size=G.WINDOW_SIZE):
    forecast = []
    for time in range(len(series) - window_size):
        forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

    forecast = forecast[split_time-window_size:]
    results = np.array(forecast)[:, 0, 0]
    return results


# Salvar a previsão
dnn_forecast = generate_forecast()

# Plote
plt.figure(figsize=(10, 6))
plot_series(time_valid, series_valid)
plot_series(time_valid, dnn_forecast)

**Saída esperada:**

Uma série semelhante a esta 

<div>
<img src="images/forecast.png" width="500"/>
</div>

In [ ]:
mse, mae = compute_metrics(series_valid, dnn_forecast)

print(f"mse: {mse:.2f}, mae: {mae:.2f} para a previsão")

**Para ser aprovado nesta tarefa, sua previsão deve atingir um MSE de 30 ou menos.**

- Se a sua previsão não atingir esse limite, tente treinar novamente o modelo com uma arquitetura diferente ou ajustar os parâmetros do otimizador.


- Se a previsão tiver atingido esse limite, execute a seguinte célula para salvar o modelo em um arquivo HDF5 que será usado para avaliação e, depois disso, envie a tarefa para avaliação.


- Certifique-se de não ter usado camadas `Lambda` em seu modelo, pois elas são incompatíveis com o formato `HDF5` que será usado para salvar seu modelo para classificação.


- Esse ambiente inclui um arquivo fictício `my_model.h5` que é apenas um modelo fictício treinado para uma época. **Para substituir esse arquivo pelo seu modelo real, é necessário executar a próxima célula antes de também enviá-lo para avaliação, junto com o notebook.**

In [ ]:
# Salve seu modelo no formato HDF5
model.save('my_model.h5')

**Parabéns por ter concluído a tarefa!**

Você implementou com sucesso uma rede neural capaz de prever séries temporais e, ao mesmo tempo, aprendeu a aproveitar a classe Dataset do Tensorflow para processar dados de séries temporais!